Процесс обучения NLP-моделей как правило состоит из двух шагов: предобучения и адаптации под какую-либо конкретную задачу. Наиболее распространенным способом адаптации является файн-тюнинг, но существуют и другие способы, такие как in-context learning. Авторы данной работы предлагают еще один способ адаптации моделей под конкретные задачи, названный P-learning. Далее сравним все перечисленные способы.

### Fine-tuning

При файн-тюнинге модель дообучается на размеченных данных для конкретной задачи. Однако авторы утверждают, что большие языковые модели (такие как GPT-3) быстро запоминают все обучающие примеры, что снижает эффективность файн-тюнинга. В целом было замечено, что GPT-подобные модели не достигают на задачах понимания естественного языка (NLU) такого же качества, как BERT-подобные модели.

> For long, researchers have observed that GPT-style models perform poorly for NLU tasks with fine-tuning, and thus assumed that they are not suitable for language understanding in nature.

Есть правда еще один метод дообучения (*feature-based*), когда для конкретной задачи создается и обучается отдельный выходной слой, при этом остальные веса модели не меняются - этот подход авторы данной работы не упоминают.

### In-context learning

Пусть наша задача - построение вопросно-ответной системы, и мы имеем N обучающих примеров из вопросов и ответов на них. Вместо того, чтобы выполнять файн-тюнинг модели с помощью этих данных, мы можем просто сформировать следующий текстовый запрос:

*"вопрос1; ответ1; ..., вопросN; ответN; вопрос"*

Последним идет вопрос, на который мы хотим получить ответ. Опционально перед примерами можно добавить также формулировку задачи в текстовом виде. Передав получившийся текст на вход модели, на выходе мы получаем ответ на него. Особенно хорошо этот способ работает для больших моделей, вроде GPT-3. В зависимости от количества примеров такой способ называется *zero-shot*, *one-shot* или *few-shot in-context learning*. Ниже приведены примеры из статьи, посвященной модели GPT-3 ([Brown et al., 2020]($Language Models are Few-Shot Learners$)).

<img src="assets/incontext.jpg" width="700" align="center">

При этом не происходит обновления весов модели градиентным спуском, поэтому такой способ не является обучением в стандартном понимании этого термина применительно к нейронным сетям, но в каком-то смысле это все же обучение. Авторы GPT-3 рассматривают такой процесс как *meta-learning* (см. [Brown et al., 2020]($Language Models are Few-Shot Learners$)).

В каких-то случаях такой способ может быть предпочтительнее, чем файн-тюнинг. Например, задачей может являться получение ответов на вопросы (расположение городов, исторические факты и пр.) - такие ответы требуют различных знаний о мире, накопленных в процессе предобучения. При файн-тюнинге веса сети меняются, при этом есть опасность, что часть знаний "сотрется" (а именно знания о тех фактах, которые не попали в обучающую выборку).

> ...in terms of knowledge probing, many facts can only be hard-coded rather than inferenced by language models. The fine-tuning of parameters might result in catastrophic forgetting.

### Manual promting

Под термином manual promting авторы данной работы понимают разновидность zero-shot in-context learning, при которой особое внимание уделяется подбору текстового описания задачи. Таким образом мы ищем наилучший способ "дать понять" модели чего мы хотим. На иллюстрации ниже показано несколько примеров текстового описания задачи, где вместо [X] подставляются исходные данные, а вместо [Y] целевые данные. Показана также точность работы такого метода для модели `bert-base-cased` на датасете из городов и стран. Можно видеть, что разные формулировки задачи дают существенно разную точность.

<img src="assets/promting.png" width="300" align="center">

Каждое описание задачи состоит из последовательности токенов. Например, для BERT-подобных моделей формат может быть следующим:

`<promt1>` [X] `<promt2>` [Y] `<promt3>`
    
Здесь `<promt1>`, `<promt2>`, `<promt3>` - токены, описывающие задачу в текстовом виде (как на иллюстрации выше), [X] - входные данные, [Y] - токен [MASK], который модель должна заменить на ответ. Здесь, правда, есть ограничение: ответ должен помещаться в один токен. Для GPT-подобных моделей формат может быть следующим:

`<promt1>` [X] `<promt2>`

Продолжение текста, сгенерированное моделью, считается ответом.

В различных работах были предложены методы автоматического подбора подходящих описаний задачи (на основе точности на валидационном датасете).

Можно объединить файн-тюнинг и manual promting, подбирая сначала оптимальное текстовое описание задачи (и таким образом задавая формат входных данных), и затем выполняя файн-тюнинг сети.

### P-tuning

Вместо того, чтобы подбирать токены, можно напрямую оптимизировать их эмбеддинги градиентным спуском - в этом и заключается метод P-tuning. При этом мы можем получить эмбеддинги, которые не соответствуют ни одному токену в словаре, но качество получаемого решения оказывается выше, чем при manual promting, что показывают авторы данной работы.

Если сравнивать P-tuning и fine-tuning, то отличие в следующем: если при файн-тюнинге оптимизируются веса сети, то при P-tuning оптимизируются дополнительные вектора-эмбеддинги, подаваемые на вход модели.

*Примечание.* Как мне кажется, для GPT-подобных моделей формат `<promt1>` [X] `<promt2>` можно без потери общности заменить на `<promt1>` [X], так как `<promt1>` и `<promt2>` отличаются только позиционными эмбеддингами, и таким образом оптимизируется сумма случайного вектора и позиционного эмбеддинга, что эквивалентно оптимизации просто случайного вектора.

Можно также провести параллель с [Lu et al., 2021]($Pretrained Transformers as Universal Computation Engines$), где предобученный трансформер с замороженными весами обучали решать различные задачи, оптимизируя подаваемые на вход вектора эмбеддинги (в этой работе оптимизировался также выходной слой трансформера и параметры слоев LayerNormalization).

Авторы предлагают еще одну модификацию (см. раздел 3.2 статьи): те вектора-эмбеддинги, которые оптимизируются в ходе P-tuning, перед подачей на вход трансформера пропускаются через BiLSTM, и оптимизируются также веса этой сети.

### Сравнение эффективности

Авторы сравнивают на задачах из бенчмарка SuperGLUE следующие подходы:
- Fine-tuning
- MP zero-shot (Manual promting)
- MP fine-tuning (Manual promting + fine-tuning)
- P-tuning

Ниже показана средняя точность по 7 задачам из бенчмарка SuperGLUE (более подробные таблицы можно найти в статье).

<img src="assets/ptuning.jpg" width="550" align="center">

Как в случае *base-sized models*, так и в случае *large-sized models* видна одна и та же закономерность: при файн-тюнинге BERT дает лучший результат, чем GPT, тогда как при P-tuning, наоборот, GPT показывает наилучший результат. При этом P-tuning улучшает результат и для модели BERT.

Manual promting + fine-tuning показывает результат, сопоставимый с P-tuning, но при этом также меняются веса модели, тогда как при P-tuning веса модели не меняются.